# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

In [101]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [102]:
# your code here
import pandas as pd
data = pd.read_csv('/home/jc/Escritorio/Github_labs/dataptmad0420/module-3/deep-learning/your-code/tic-tac-toe.csv')

In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
TL       958 non-null object
TM       958 non-null object
TR       958 non-null object
ML       958 non-null object
MM       958 non-null object
MR       958 non-null object
BL       958 non-null object
BM       958 non-null object
BR       958 non-null object
class    958 non-null bool
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [104]:
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [105]:
data.shape

(958, 10)

In [106]:
data.columns.values.tolist()

['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class']

In [107]:
X = data.iloc[:, 0:9].values
y = data.iloc[:, 9:10].values

In [108]:
labelencoder_X = LabelEncoder()
for _ in range(9):
    X[:, _] = labelencoder_X.fit_transform(X[:, _])

In [109]:
X

array([[2, 2, 2, ..., 2, 1, 1],
       [2, 2, 2, ..., 1, 2, 1],
       [2, 2, 2, ..., 1, 1, 2],
       ...,
       [1, 2, 1, ..., 2, 1, 2],
       [1, 2, 1, ..., 2, 1, 2],
       [1, 1, 2, ..., 1, 2, 2]], dtype=object)

In [110]:
onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(X).toarray()


In [111]:
X = np.delete(X, [0,3,6,9,12,15,18,21,24], axis=1)

In [112]:
labelencoder_y = LabelEncoder()
y[:, 0] = labelencoder_y.fit_transform(y[:, 0])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [138]:
# your code here

# Initialize neural network
nnet = Sequential()

# Add first hidden layer (and input layer)
nnet.add(Dense(units=9, kernel_initializer='uniform', activation='relu', input_dim=18))



# Add output layer
nnet.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Compile network
nnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train network
nnet.fit(X_train, y_train, batch_size=10, epochs=50)

# Predicting the test set results
y_pred = nnet.predict(X_test)




Epoch 1/50
766/766 [==============================] - 0s 386us/step - loss: 0.6777 - accuracy: 0.6540
Epoch 2/50
766/766 [==============================] - 0s 171us/step - loss: 0.6390 - accuracy: 0.6540
Epoch 3/50
766/766 [==============================] - 0s 167us/step - loss: 0.6083 - accuracy: 0.6554
Epoch 4/50
766/766 [==============================] - 0s 176us/step - loss: 0.5819 - accuracy: 0.7076
Epoch 5/50
766/766 [==============================] - 0s 173us/step - loss: 0.5599 - accuracy: 0.7363
Epoch 6/50
766/766 [==============================] - 0s 186us/step - loss: 0.5445 - accuracy: 0.7415
Epoch 7/50
766/766 [==============================] - 0s 189us/step - loss: 0.5301 - accuracy: 0.7611
Epoch 8/50
766/766 [==============================] - 0s 172us/step - loss: 0.5192 - accuracy: 0.7572
Epoch 9/50
766/766 [==============================] - 0s 159us/step - loss: 0.5085 - accuracy: 0.7572
Epoch 10/50
766/766 [==============================] - 0s 165us/step - loss: 0.497

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [131]:
# your code here

print(type(X_train))
print(X_train[0])
print(len(X_train[0]))

print(X_train[0].shape)
print(y_train[0])


y_pred_1 = nnet.predict(X_train[0:18])
y_pred_2 = nnet.predict(X_train[19:37])

print(y_pred_1)
print(y_pred_2)



<class 'numpy.ndarray'>
[1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
18
(18,)
[ True]
[[9.9799722e-01]
 [9.9830413e-01]
 [9.9993062e-01]
 [9.9988592e-01]
 [9.9781466e-01]
 [9.9393940e-01]
 [9.9987125e-01]
 [6.0692430e-04]
 [9.9691737e-01]
 [9.9739110e-01]
 [9.9760240e-01]
 [9.9656010e-01]
 [9.5458400e-01]
 [9.9991477e-01]
 [9.9993688e-01]
 [9.9750000e-01]
 [9.4107217e-01]
 [9.9766552e-01]]
[[9.9649358e-01]
 [9.9591410e-01]
 [9.9682367e-01]
 [9.9597579e-01]
 [9.9375272e-01]
 [3.8203597e-04]
 [4.1601062e-04]
 [9.9984372e-01]
 [4.2876601e-04]
 [9.9845821e-01]
 [3.9616227e-04]
 [9.9988669e-01]
 [9.9836981e-01]
 [4.1237473e-04]
 [7.0916712e-03]
 [9.9990022e-01]
 [9.9722505e-01]
 [4.6282809e-04]]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

# your answer here

The nn learns better with a second hidden layer

In [135]:
# your code here

# Initialize neural network
nnet = Sequential()

# Add first hidden layer (and input layer)
nnet.add(Dense(units=9, kernel_initializer='uniform', activation='relu', input_dim=18))

# Add second hidden layer
nnet.add(Dense(units=9, kernel_initializer='uniform', activation='relu'))

# Add third hidden layer
nnet.add(Dense(units=9, kernel_initializer='uniform', activation='relu'))

# Add output layer
nnet.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Compile network
nnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train network
nnet.fit(X_train, y_train, batch_size=10, epochs=100)

# Predicting the test set results
y_pred = nnet.predict(X_test)

Epoch 1/100
766/766 [==============================] - 0s 553us/step - loss: 0.6875 - accuracy: 0.6501
Epoch 2/100
766/766 [==============================] - 0s 184us/step - loss: 0.6543 - accuracy: 0.6540
Epoch 3/100
766/766 [==============================] - 0s 193us/step - loss: 0.6154 - accuracy: 0.6540
Epoch 4/100
766/766 [==============================] - 0s 185us/step - loss: 0.5897 - accuracy: 0.6540
Epoch 5/100
766/766 [==============================] - 0s 183us/step - loss: 0.5744 - accuracy: 0.6540
Epoch 6/100
766/766 [==============================] - 0s 194us/step - loss: 0.5609 - accuracy: 0.6540
Epoch 7/100
766/766 [==============================] - 0s 214us/step - loss: 0.5516 - accuracy: 0.6540
Epoch 8/100
766/766 [==============================] - 0s 217us/step - loss: 0.5419 - accuracy: 0.7441
Epoch 9/100
766/766 [==============================] - 0s 192us/step - loss: 0.5302 - accuracy: 0.7663
Epoch 10/100
766/766 [==============================] - 0s 205us/step - l